In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from pmlb import fetch_data

from pdpilot import partial_dependence, PDPilotWidget

In [ ]:
df = fetch_data('churn')

In [ ]:
df

In [ ]:
df_X = df.drop(columns=[
    'target', 'state', 'phone number',
    'total day charge', 'total night charge',
    'total eve charge'
])
y = df['target'].values

In [ ]:
regr = RandomForestRegressor(n_estimators=100, max_features='sqrt')
regr.fit(df_X, y)

In [ ]:
features = list(df_X.columns)

In [ ]:
subset = df_X.sample(1000)
labels = y[subset.index].tolist()

In [ ]:
pd_data = partial_dependence(
    predict=regr.predict,
    df=subset,
    features=features,
    ordinal_features={'international plan', 'voice mail plan', 'number customer service calls'},
    nominal_features={'area code'},
    resolution=20,
    n_jobs=4,
)

In [ ]:
w = PDPilotWidget(
    predict=regr.predict,
    df=subset,
    labels=labels,
    pd_data=pd_data,
    height=650
)

w